# Aswesome Oscillator

In [ ]:
# trading_system/scripts/run_ao_optimization.py
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Awesome Oscillator strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.momentum.awesome_oscillator_strat import AwesomeOscillatorStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)


In [ ]:
# single stock

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all


# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 3
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Awesome Oscillator ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import awesome_oscillator_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Awesome Oscillator Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

        # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"AwesomeOscillator_Optimization_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {AwesomeOscillatorStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=AwesomeOscillatorStrategy,
            db_config=db_config,
            search_space=awesome_oscillator_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"AwesomeOscillator_Opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Coppock Curve

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Coppock Oscillator strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.momentum.coppock_curve_strat import CoppockCurveStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for Coppock ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import coppock_curve_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Coppock Oscillator Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

    # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"AwesomeOscillator_Optimization_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {CoppockCurveStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=CoppockCurveStrategy,
            db_config=db_config,
            search_space=coppock_curve_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"CoppockOscillator_Opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Run Just the Strat

In [ ]:
# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)

db_config = DatabaseConfig.default()

cc = CoppockCurveStrategy(db_config, params = {'long_only': True, 'method': 'zero_crossing', 'normalize_strength': True, 'roc1_months': 10, 'roc2_months': 8, 'stop_loss_pct': 0.11588214818083328, 'strength_window': 378, 'sustain_days': 6, 'take_profit_pct': 0.06368548764576594, 'trailing_stop_pct': 0.04324839596707414, 'trend_strength_threshold': 0.7830082690475033, 'wma_lookback': 14, 'slippage_pct': 0.001, 'transaction_cost_pct': 0.001})

a = cc.generate_signals(ticker=tickers_to_run, start_date=START_DATE, end_date=END_DATE)

In [ ]:
a.signal.value_counts()

In [ ]:
b = a[a.signal == 1]

In [ ]:
b.groupby('ticker').size().reset_index(name='counts').sort_values(by='counts', ascending=False)

# Know Sure Thing

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Know Sure Thing strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.momentum.know_sure_thing_strat import KSTStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:
# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for KST ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import know_sure_thing_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Know Sure Thing Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

    # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"AwesomeOscillator_Optimization_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {KSTStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=KSTStrategy,
            db_config=db_config,
            search_space=know_sure_thing_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"kst_opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# MCAD

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Know Sure Thing strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.momentum.macd_strat import MACDStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = 10 # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for KST ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import mcad_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting Know Sure Thing Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

    # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"MACD_Optimization_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {MACDStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class=MACDStrategy,
            db_config=db_config,
            search_space=mcad_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"macd_opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# Just Raw code

In [ ]:
# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)

db_config = DatabaseConfig.default()

cc = CoppockCurveStrategy(db_config, params = {'long_only': True, 'method': 'zero_crossing', 'normalize_strength': True, 'roc1_months': 10, 'roc2_months': 8, 'stop_loss_pct': 0.11588214818083328, 'strength_window': 378, 'sustain_days': 6, 'take_profit_pct': 0.06368548764576594, 'trailing_stop_pct': 0.04324839596707414, 'trend_strength_threshold': 0.7830082690475033, 'wma_lookback': 14, 'slippage_pct': 0.001, 'transaction_cost_pct': 0.001})

a = cc.generate_signals(ticker=tickers_to_run, start_date=START_DATE, end_date=END_DATE)

# Relative Vigor Index

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Know Sure Thing strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.momentum.relative_vigor_index_strat import RVIStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:
# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for MACD ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import relative_vigor_index_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting RVI Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)


    # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"MACD_Optimization_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {RVIStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class= RVIStrategy,
            db_config=db_config,
            search_space=relative_vigor_index_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"kst_opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# SMA Crossover

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Know Sure Thing strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.momentum.sma_crossover_strat import SMAStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:
# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for MACD ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import sma_crossover_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting RVI Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)

    # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"SMA_Optimization_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {SMAStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class= SMAStrategy,
            db_config=db_config,
            search_space=sma_crossover_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"kst_opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")

# TSI Strategy

In [ ]:
"""
Main script to run hyperparameter optimization and sensitivity analysis
for the Know Sure Thing strategy using the portfolio-based evaluation framework.
"""
import json
import logging
import os
import sys
from datetime import datetime, timedelta

import mlflow
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    from src.strategies.momentum.true_strength_index_strat import TSIStrategy
    from src.optimizer.strategy_optimizer import StrategyOptimizer
    from src.optimizer.sensitivity_analyzer import SensitivityAnalyzer
    from src.database.config import DatabaseConfig
    from utils.file_utils import load_tickers_from_yaml
except ImportError as e:
    print("Error importing modules. Make sure the script is run from the project root")
    print("or the 'src' directory is in the Python path.")
    print(f"Import Error: {e}")
    sys.exit(1)

In [ ]:
# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# MLflow Configuration
MLFLOW_TRACKING_URI = "file:./mlruns"  # Store MLflow data locally in ./mlruns
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Data Configuration
TICKER_FILE_PATH = "../data/tickers.yml" # Path relative to project root
MAX_TICKERS = None # Limit tickers for faster testing, set to None to use all

# Backtest Period
START_DATE = (datetime.now() - timedelta(days=4*365)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Optimization Settings
CV_FOLDS = 5
MAX_EVALS = 50  # Number of hyperparameter sets to evaluate
OPTIMIZATION_METRIC = 'harmonic_mean' # Portfolio metric to maximize (minus penalty)
N_JOBS = -1 # Use all available CPU cores for fold evaluation within optimizer

# Sensitivity Analysis Settings
RUN_SENSITIVITY = False # Set to False to skip sensitivity analysis
NUMERIC_PERTURBATION = 0.15 # +/- 15% for sensitivity
SENS_SAMPLES_PER_PARAM = 5
SENS_RANDOM_SAMPLES = 20

# --- Define Search Space for MACD ---

# Note: Hyperopt doesn't easily enforce short_period < long_period directly during sampling.
# The optimizer will evaluate invalid combinations, and they will likely fail or perform poorly.
# Strategy itself raises ValueError if short >= long during initialization.

from src.optimizer.search_space import true_strength_index_strat_search_space

# --- Main Execution ---

if __name__ == "__main__":
    logger.info("--- Starting RVI Optimization Script ---")

    # Setup MLflow
    try:
        mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        logger.info(f"MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")
    except Exception as e:
        logger.error(f"Failed to set MLflow tracking URI: {e}")
        sys.exit(1)


    # Create MLflow experiment if it doesn't exist
    try:
        experiment_name = f"TSI_Optimization_{RUN_TIMESTAMP}"
        # Check if experiment exists
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Create new experiment
            experiment_id = mlflow.create_experiment(experiment_name)
            logger.info(f"Created new MLflow experiment: {experiment_name} with ID: {experiment_id}")
        else:
            experiment_id = experiment.experiment_id
            logger.info(f"Using existing MLflow experiment: {experiment_name} with ID: {experiment_id}")
        
        # Set the experiment for subsequent runs
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        logger.error(f"Failed to create or set MLflow experiment: {e}")
        sys.exit(1)

    # Load Tickers
    try:
        tickers_to_run = load_tickers_from_yaml(TICKER_FILE_PATH, MAX_TICKERS)
    except Exception:
        logger.error("Failed to load tickers. Exiting.")
        sys.exit(1)

    # Database Config
    try:
        db_config = DatabaseConfig.default()
        # Optional: Add a check here to ensure DB connection is valid if possible
        logger.info("Database configuration loaded.")
    except Exception as e:
        logger.error(f"Failed to load database configuration: {e}")
        sys.exit(1)

    # --- Run Optimization ---
    optimizer = None
    best_params = {}
    portfolio_performance_report = pd.DataFrame()
    param_history_report = pd.DataFrame()

    logger.info(f"Initializing StrategyOptimizer for {TSIStrategy.__name__}")
    try:
        optimizer = StrategyOptimizer(
            strategy_class= TSIStrategy,
            db_config=db_config,
            search_space=true_strength_index_strat_search_space,
            tickers=tickers_to_run,
            start_date=START_DATE,
            end_date=END_DATE,
            cv_folds=CV_FOLDS,
            max_evals=MAX_EVALS,
            optimization_metric=OPTIMIZATION_METRIC,
            run_name=f"kst_opt_{RUN_TIMESTAMP}",
            n_jobs=N_JOBS
            # risk_thresholds can be customized here if needed, otherwise defaults are used
        )

        logger.info("Starting hyperparameter optimization...")
        best_params, portfolio_performance_report, param_history_report = optimizer.run_optimization()

        if not best_params:
             logger.error("Optimization did not yield valid results. Best parameters not found.")
        else:
             logger.info("--- Optimization Results ---")
             logger.info(f"Best Parameters found:\n{json.dumps(best_params, indent=2)}")
             logger.info(f"\nBest Portfolio Performance Report:\n{portfolio_performance_report.to_string()}")
             logger.info(f"\nParameter History saved (see MLflow artifacts or CSV file). Head:\n{param_history_report.head().to_string()}")

    except Exception as e:
        logger.error(f"An error occurred during optimization: {e}", exc_info=True)
        # Attempt to end MLflow run if it was started by the optimizer
        if mlflow.active_run():
            mlflow.end_run("FAILED")

    # --- Run Sensitivity Analysis (Optional) ---
    if RUN_SENSITIVITY and optimizer and best_params:
        logger.info("\n--- Starting Sensitivity Analysis ---")
        try:
            analyzer = SensitivityAnalyzer(
                strategy_optimizer=optimizer, # Reuse optimizer for its config and evaluation cache
                base_params=best_params,
                numeric_perturbation=NUMERIC_PERTURBATION,
                num_samples_per_param=SENS_SAMPLES_PER_PARAM,
                num_random_samples=SENS_RANDOM_SAMPLES,
                parallel=True # Relies on optimizer's internal parallelization/caching
            )

            sensitivity_results_df, parameter_impact_df = analyzer.run()

            if sensitivity_results_df.empty:
                 logger.warning("Sensitivity analysis did not produce results.")
            else:
                logger.info("--- Sensitivity Analysis Results ---")
                logger.info(f"Sensitivity Results saved (see MLflow artifacts or CSV file). Head:\n{sensitivity_results_df.head().to_string()}")
                logger.info(f"\nParameter Impact Report (Correlation):\n{parameter_impact_df.to_string()}")

        except Exception as e:
            logger.error(f"An error occurred during sensitivity analysis: {e}", exc_info=True)
            if mlflow.active_run():
                 mlflow.end_run("FAILED") # End sensitivity run if it crashed

    elif RUN_SENSITIVITY and (not optimizer or not best_params):
        logger.warning("Skipping sensitivity analysis because optimization failed or produced no best parameters.")


    # Ensure any lingering run is terminated cleanly
    # Should not be necessary if 'with mlflow.start_run()' is used correctly inside modules
    # try:
    #     while mlflow.active_run():
    #         logger.info(f"Ending lingering MLflow run: {mlflow.active_run().info.run_id}")
    #         mlflow.end_run()
    # except Exception:
    #      pass # Ignore errors during cleanup

    logger.info("--- Script Finished ---")